In [986]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [987]:
matches = pd.read_csv("assets/2023-2024.csv")

In [988]:
matches["Date Time (US Eastern)"] = pd.to_datetime(matches["Date Time (US Eastern)"])

""" matches["Date (US Eastern)"] = pd.to_datetime(matches["Date (US Eastern)"])
matches["Time (US Eastern)"] = pd.to_datetime(matches["Time (US Eastern)"]) """

/var/folders/cn/j21vzthj6ds5gjqdtwlgj4f00000gn/T/ipykernel_1051/1605438902.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  matches["Date Time (US Eastern)"] = pd.to_datetime(matches["Date Time (US Eastern)"])


' matches["Date (US Eastern)"] = pd.to_datetime(matches["Date (US Eastern)"])\nmatches["Time (US Eastern)"] = pd.to_datetime(matches["Time (US Eastern)"]) '

In [989]:
#advantages and disadvantages to consider when teams play against each other

matches["homeAway_code"] = matches["homeAway"].astype("category").cat.codes

matches["opponent_code"] = np.where(
    matches["Team"] == matches["Home Team"],
    matches["Away Team"].astype("category").cat.codes,
    matches["Home Team"].astype("category").cat.codes
)
matches["hour"] = matches["Date Time (US Eastern)"].dt.hour.astype("Int64")
matches["day_code"] = matches["Date Time (US Eastern)"].dt.dayofweek
matches["date"] = matches["Date Time (US Eastern)"].dt.date

""" 
info to train the model
2. On target shots
3. Goal (expected goals)
4. Possession 
5. Pass completsion 
"""
matches

,Team,Date Time (US Eastern),Fixture,Home Team,Home Goal,Away Goal,Away Team,homeAway,foulsCommitted,yellowCards,...,interceptions,effectiveClearance,totalClearance,updateTime,Unnamed: 37,homeAway_code,opponent_code,hour,day_code,date
0,Burnley,2023-08-11 15:00:00,Manchester City at Burnley,Burnley,0.0,3.0,Manchester City,home,11.0,0.0,...,7.0,16.0,16.0,5/20/24 16:45,NaN,1,12,15,4.0,2023-08-11
1,Manchester City,2023-08-11 15:00:00,Manchester City at Burnley,Burnley,0.0,3.0,Manchester City,away,8.0,0.0,...,4.0,6.0,6.0,5/20/24 16:45,NaN,0,5,15,4.0,2023-08-11
2,Arsenal,2023-08-12 08:00:00,Nottingham Forest at Arsenal,Arsenal,2.0,1.0,Nottingham Forest,home,12.0,2.0,...,7.0,9.0,9.0,5/20/24 16:45,NaN,1,15,8,5.0,2023-08-12
3,Nottingham Forest,2023-08-12 08:00:00,Nottingham Forest at Arsenal,Arsenal,2.0,1.0,Nottingham Forest,away,12.0,2.0,...,8.0,16.0,16.0,5/20/24 16:45,NaN,0,1,8,5.0,2023-08-12
4,AFC Bournemouth,2023-08-12 10:00:00,West Ham United at AFC Bournemouth,AFC Bournemouth,1.0,1.0,West Ham United,home,9.0,1.0,...,7.0,10.0,10.0,5/20/24 16:45,NaN,1,18,10,5.0,2023-08-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-1,-1,<NA>,NaN,NaT
1100,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-1,-1,<NA>,NaN,NaT
1101,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-1,-1,<NA>,NaN,NaT
1102,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-1,-1,<NA>,NaN,NaT


In [990]:
#determine the match result
matches["target"] = np.where(
    matches["Home Goal"] == matches["Away Goal"], 2,  
    np.where(
        (matches["Home Goal"] > matches["Away Goal"]) & (matches["Team"] == matches["Home Team"]), 1,  
        np.where(
            (matches["Home Goal"] > matches["Away Goal"]) & (matches["Team"] != matches["Home Team"]), 0,  
            np.where(
                (matches["Home Goal"] < matches["Away Goal"]) & (matches["Team"] == matches["Away Team"]), 1,  
                0  
            )
        )
    )
)

In [991]:
#began using machine learning 
rf = RandomForestClassifier(n_estimators=100, min_samples_split=20, random_state=42)

In [992]:
predictors = ["homeAway_code", "opponent_code", "hour", "day_code"]
""" p = [   "homeAway_code", "opponent_code", "hour", "day_code", 
        "shotsOnTarget", "accuratePasses", "accurateCrosses", 
        "accurateLongBalls", "blockedShots", "effectiveTackles", "effectiveClearance"
    ] """

' p = [   "homeAway_code", "opponent_code", "hour", "day_code", \n        "shotsOnTarget", "accuratePasses", "accurateCrosses", \n        "accurateLongBalls", "blockedShots", "effectiveTackles", "effectiveClearance"\n    ] '

In [993]:
train = matches[matches["Date Time (US Eastern)"] <= '2024-01-01']

In [994]:
test = matches[matches["Date Time (US Eastern)"] > '2024-01-01']

In [995]:
rf.fit(train[predictors], train["target"])
#rf.fit(train[p], train["target"])

RandomForestClassifier(min_samples_split=20, random_state=42)

In [996]:
preds = rf.predict(test[predictors])
#preds = rf.predict(test[p])

In [997]:
accuracy = accuracy_score(test["target"], preds)
accuracy

0.3967391304347826

In [998]:
combine = pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [999]:
pd.crosstab(index=combine["actual"], columns=combine["prediction"])

prediction,0,1,2
actual,,,
0,71,58,9
1,59,72,7
2,45,44,3


In [1000]:
precision_score(test["target"], preds, average='macro', zero_division=0)

0.32580070866822225

In [1001]:
grouped_matches = matches.groupby("Team")
""" group = grouped_matches.get_group("West Ham United")
group """

' group = grouped_matches.get_group("West Ham United")\ngroup '

In [1002]:
""" 
This function uses the stats from the previous 3 games 
to determine the team's condition and predict the next game. 
"""
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("Date Time (US Eastern)")
    rolling_stats = group[cols].shift(1).rolling(window=3).mean()
    for col, new_col in zip(cols, new_cols):
            group[new_col] = rolling_stats[col]
    group = group.dropna(subset=new_cols)
    return group

In [1003]:
columns = [
    "totalShots", "shotsOnTarget", 
    "possessionPct", "wonCorners", 
    "accuratePasses", "totalPasses", 
    "effectiveTackles", "totalTackles",
    ]
new_columns = [f"{c}_rolling" for c in columns]

In [1004]:
""" rolling_averages(group, columns, new_columns) """
matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, columns, new_columns))
matches_rolling.droplevel("Team")
matches_rolling.index = range(matches_rolling.shape[0])
#matches_rolling

/var/folders/cn/j21vzthj6ds5gjqdtwlgj4f00000gn/T/ipykernel_1051/2647377087.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, columns, new_columns))


In [1005]:
def make_predictions(data, predictors):
    train = data[data["Date Time (US Eastern)"] <= '2024-01-01']
    test = data[data["Date Time (US Eastern)"] > '2024-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combine = pd.DataFrame(dict(actual=test["target"], prediction=preds), index=test.index)
    precision = precision_score(test["target"], preds, average='macro', zero_division=0)
    return combine, precision

In [1006]:
combined, precision = make_predictions(matches_rolling, predictors + new_columns)

combined = combined.merge(matches_rolling[["date", "Home Team", "Away Team", "target"]], left_index=True, right_index=True)
combined


,actual,prediction,date,Home Team,Away Team,target
16,0,1,2024-01-21,AFC Bournemouth,Liverpool,0
17,2,0,2024-02-01,West Ham United,AFC Bournemouth,2
18,2,0,2024-02-04,AFC Bournemouth,Nottingham Forest,2
19,0,0,2024-02-10,Fulham,AFC Bournemouth,0
20,2,0,2024-02-17,Newcastle United,AFC Bournemouth,2
...,...,...,...,...,...,...
695,0,0,2024-04-24,Wolverhampton Wanderers,AFC Bournemouth,0
696,1,1,2024-04-27,Wolverhampton Wanderers,Luton Town,1
697,0,0,2024-05-04,Manchester City,Wolverhampton Wanderers,0
698,0,1,2024-05-11,Wolverhampton Wanderers,Crystal Palace,0


In [1007]:
precision

0.6618637353931471

In [1008]:
accuracy

0.3967391304347826